In [57]:
from sklearn.datasets import load_boston
from sklearn.metrics import explained_variance_score
import numpy as np

x = load_boston().data
y = load_boston().target

In [58]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=29)

x_train.shape

(379, 13)

In [59]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [60]:
# scaling
def scaling(X):
    for c in range(X.shape[1]):
        X[:,c] = [(i - np.mean(X[:,c])) / (max(X[:,c]) - min(X[:,c])) for i in X[:,c]]
    return X

# normalize
def normal(X):
    return X / X.max(axis=0)

In [61]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), normal(X)] @ theta

In [62]:
def normalEq(X, y):
    X = np.c_[np.ones(len(X)), normal(X)]
    return (np.linalg.inv(X.T @ X) @ X.T @ y).T

theta = normalEq(x_train, y_train)

print(theta)

preds = predict(x_test, theta)

print(f'\nNorm Eq EVS: {explained_variance_score(preds, y_test)}')

[ 35.98189827  -6.88140442   5.31800698   1.01840583   3.43338847
 -12.89840309  31.08338357  -0.25493095 -16.56781587   7.73387839
  -9.19866233 -21.23376392   4.39090559 -20.92958659]

Norm Eq EVS: 0.717097886997718


$X^{'}X=\begin{bmatrix} 7 & 38.5\\  38.5& 218.75 \end{bmatrix}$

In [80]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(polynom(x_train, 2), y_train, 1000, 0.001)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\nBGD EVS: {explained_variance_score(preds, y_test)}')

[[  2.20223782e+01  -5.19784076e+00   4.83770305e-01  -2.59776506e+00
    2.12002088e+00   1.24695326e+00   1.14681504e+01  -3.91248137e+00
   -8.95968720e+00   6.67506125e+00  -4.10215510e+00  -1.84776717e+00
    6.27458933e+00  -2.62575013e+01  -1.77267044e+00   4.43135725e+00
    4.29923131e+00   1.39102456e+00  -7.58538291e+00   2.04871472e+01
    3.20466708e+00  -2.94885416e+00   6.09839535e-03  -3.75989400e+00
   -9.55363975e+00  -1.12117842e+00   1.04071540e+01]]

BGD EVS: 0.7411088254882143


In [82]:
import time

start = time.clock()

def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[r:r+1], y.T[r:r+1]
                gradients = ((xi @ theta.T) - yi.T).T @ xi[:,c]
#                 eta = eta/epoch
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T

theta = SGD(polynom(x_train, 2), y_train, 50, 0.01)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\nSGD EVS: {explained_variance_score(preds, y_test)}')

print(time.clock() - start)

[[  4.18379233  -3.71860706   3.821575     2.00446984   1.15910126
    4.75921166  16.92737301   6.18395939  -3.80336233  -0.55577738
   -1.08128133   4.28690803  13.50923699 -19.96798789  -0.93936281
    1.38009989   0.62542377   2.48007316  -6.75509392  19.43180909
   -5.18424831  -4.78353346   5.70475352  -4.88503682 -12.04338514
   -6.53109802   3.77022403]]

SGD EVS: 0.6933477276102831
3.8847449999999952


In [83]:
import time

start = time.clock()

def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(polynom(x_train, 2), y_train, 500, 64, 0.01)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\n MBGD EVS: {explained_variance_score(preds, y_test)}')

print(time.clock() - start)

[[ 36.87369548 -16.80812274  -3.88168439  -9.38996353   5.99668675
    2.86621843  -3.09339067  -5.8625074  -17.20407428  11.00759676
   -5.17865601   5.06277094  12.53436256 -55.61069993   8.31489337
    7.37282102  10.87291703  -2.66597745 -10.44962978  23.88801476
    5.89182077   4.79976558  -1.6356173   -3.27706993 -13.73666378
   -7.50788395  38.95108533]]

 MBGD EVS: 0.7546838158863626
0.5968079999999958


In [127]:
import numpy.linalg as LA

def Ridge(X, y, n_epoch, batch_size, alpha):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    print(m)
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - alpha * (gradients + (alpha * LA.norm(theta)**2))
    return theta.T

theta = Ridge(polynom(x_train, 2), y_train, 50, 64, 0.01)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\n Ridge EVS: {explained_variance_score(preds, y_test)}')

379
[[ 176.48982185 -178.22074539 -102.15676631  -68.87898967  -95.04262229
   -17.26581437   -2.80856175   -6.00922379 -127.75400668  -13.07464471
    22.39615007   39.12121671    2.20626544  -95.3582948  -217.35031848
  -137.19599997 -113.2261404  -100.92324824  -67.33013198  -44.84284871
     7.93134902 -175.12746477  -27.16227836  -12.07985465   19.11162755
    34.88942765 -138.19320119]]

 Ridge EVS: 0.055707698338447686


In [118]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [119]:
m = Ridge(alpha = 10)

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

print('weights: ')
print(m.coef_)
print('Intercept: ')
print(m.intercept_)

print(f'\n Scikit learn ridge reg EVS: {explained_variance_score(y_test, preds)}')

weights: 
[ -4.45795003e-01  -4.20281709e-02  -1.13638279e-01   1.26681724e+00
  -1.43233686e+00  -2.18378528e+00  -3.29550428e-02  -1.37782981e+00
   5.43332742e-01  -3.16984967e-02  -1.81378244e+00   2.67667275e-02
  -1.80841526e+00   4.67544909e-03   7.78662251e-04   3.97695328e-03
   1.26681724e+00  -1.98542113e+00   4.07584210e-01   3.40675265e-04
   4.48095854e-02  -7.28036088e-03   2.27704613e-05   3.08977838e-02
  -3.93823629e-05   3.58743805e-02]
Intercept: 
66.143471914

 Scikit learn ridge reg EVS: 0.7704624730353917
